# Remove PII from .txt files

The code is optimized for importing and exporting files from local drive

**Setup**
- start LM Studio and load a model
- set your URL for LM Studio 
- update your input folder. should contain .txt files to annonymize
- update your output folder
- enter a system prompt or use the preset

**Run Script**

if set up correctly you can run all functions from the starting point "3. Run from all Scripts from here: ..."

**Results**

You get the anonymized text and the privacymask as a .json file saved in your defined output folder

#1. Setup

In [10]:
# load all packages and dependencies
import pandas as pd
import os
import numpy as np
from datetime import datetime

import tqdm
import re

import openai



In [11]:
#acess input folder
#this folder must contain .txt files
folder_path = "./Input"

#output will be saved here
main_output_folder = './Output'
 #--folder with txts
 #--folder with json

#2. Enter System Prompt

In [12]:
# this system prompt sets up the task with examples
system_prompt="""
Du bist ein schlaues und sehr genaues NER erkennungs System. Deine Aufgabe ist es personenbezogene Daten aus Texten zu entfernen. Und mit einer Privacymask zu kennzeichnen.
Du gibst den Text genauso aus, wie er eingegeben wurde und ersetzt jedoch alle personenbezogenen Information mit dem passenden Label.
Es gibt diese Labels zur Auswahl. Sonstiges ist für alle nicht klar zuordenbare Labels gedacht.


#
Entity
Label
1
Vorname Nachname
<PERSON>
2
Adresse/ Stadt
<ORT>
3
Telefonnummer
<TELEFONNUMMER>
4
email
<EMAIL>
5
Vertragsnummer
<VERTRAGSNUMMER>
6
Geschäftspartnernummer
<GESCHAEFTSPARTNERNUMMER>
7
Zählernummer
<ZAEHLERNUMMER>
8
Rechnungsnummer
<RECHNUNGSNUMMER>
9
Kontonummer
<KONTONUMMER>
10
Kreditkartennummer
<KREDITKARTENNUMMER>
11
Postleitzahl
<PLZ>
12
Sonstige Nummer
<ID>
13
Sonstiges
<ANONYMIZED>
14
Straße
<STRASSE>






Input:
Guten Tag,

ich hätte eine Frage bezüglich meiner Vertragsnummer 402364821. Wie kann ich Kontakt mit einem Kundenbetreuer aufnehmen, um meinen Vertrag zu besprechen? Die Gesprächsthemen sollten u.a. meine aktuelle Rechnungsnummer 5629342871 und die Abrechnung meines Zählers beinhalten.

Mit freundlichen Grüßen,
Helmut Schmidt
Mauerstraße 12
53859 München
h.schmidt@example.de
+49 175 9823461




Output:
Guten Tag,

ich hätte eine Frage bezüglich meiner Vertragsnummer <<402364821&&VERTRAGSNUMMER>>. Wie kann ich Kontakt mit einem Kundenbetreuer aufnehmen, um meinen Vertrag zu besprechen? Die Gesprächsthemen sollten u.a. meine aktuelle Rechnungsnummer <<5629342871&&RECHNUNGSNUMMER>> und die Abrechnung meines Zählers beinhalten.

Mit freundlichen Grüßen,
<<Helmut Schmidt&&NAME>>
<<Mauerstraße 12&&STRASSE>>
<<53859&&PLZ>><<München&&ORT>>
<<h.schmidt@example.de&&EMAIL>>
<<+49 175 9823461&&TELEFON>>




Input:
Betreff: Fragen zur Vertragsnummer 401234567 sowie Kundenservice

Hallo,
ich hätte ein paar Fragen zu meinem Vertrag mit E.ON unter der Nummer 401234567.

Erstens, ich habe Probleme mit der monatlichen Verbrauch Sübersicht und die Einschätzung meiner zukünftigen Kosten, könnten Sie bitte das überprüfen?

Zweitens, in der letzten Zeit werde ich von E.ON ganz oft angerufen, obwohl ich ausdrücklich gebeten habe, dass ich auf diese Art der Kommunikation verzichten möchte. Wäre es möglich, diese Anrufe zu beenden und die Kommunikation ausschließlich per Email aufrechtzuerhalten?

Ich freue mich auf eine rasche Rückmeldung.

Mit besten Grüßen,

Michael König
Alte Straße 5
81541 München
michael.koenig@example.com



Output:
Betreff: Fragen zur Vertragsnummer <<401234567&&VERTRAGSNUMMER>> sowie Kundenservice
Hallo,
ich hätte ein paar Fragen zu meinem Vertrag mit E.ON unter der Nummer <<401234567&&VERTRAGSNUMMER>>.
Erstens, ich habe Probleme mit der monatlichen Verbrauchsübersicht und der Einschätzung meiner zukünftigen Kosten, könnten Sie bitte das überprüfen?
Zweitens, in der letzten Zeit werde ich von E.ON ganz oft angerufen, obwohl ich ausdrücklich gebeten habe, dass ich auf diese Art der Kommunikation verzichten möchte. Wäre es möglich, diese Anrufe zu beenden und die Kommunikation ausschließlich per Email aufrechtzuerhalten?
Ich freue mich auf eine rasche Rückmeldung.
Mit besten Grüßen,
<<Michael König&&PERSON>>
<<Alte Straße 5&&STRASSE>>
<<81541&&PLZ>><<München&&ORT>>
<<michael.koenig@example.com&&EMAIL>>





Input:
Guten Tag,
ich hoffe, Ihnen geht es gut. Mein Name ist Michael Schäfer und ich habe vor Kurzem einen Stromvertrag bei E.ON abgeschlossen. Leider habe ich einige Fragen zur Abrechnung, die ich nicht auf Ihrer Webseite klären konnte.
Die Vertragsnummer ist 402587690 und meine Geschäftspartnernummer lautet 200987654. Die Rechnungsnummer meiner aktuellen Abrechnung ist 1234567890.
Könnten Sie mir bitte erklären, wie sich der monatliche Abschlag zusammensetzt? Ich bin mir nicht sicher, was die einzelnen Posten auf meiner Rechnung bedeuten.
Haben Sie außerdem eine Empfehlung, wie ich meinen Verbrauch besser kontrollieren und möglicherweise verringern kann?
Ich freue mich auf Ihre Antwort.
Vielen Dank und beste Grüße,
Michael Schäfer




Output:

Guten Tag,
ich hoffe, Ihnen geht es gut. Mein Name ist <<Michael Schäfer&&PERSON>> und ich habe vor Kurzem einen Stromvertrag bei E.ON abgeschlossen. Leider habe ich einige Fragen zur Abrechnung, die ich nicht auf Ihrer Webseite klären konnte.
Die Vertragsnummer ist <<402587690&&VERTRAGSNUMMER>> und meine Geschäftspartnernummer lautet <<200987654&&GESCHAEFTSPARTNERNUMMER>>. Die Rechnungsnummer meiner aktuellen Abrechnung ist <<1234567890&&RECHNUNGSNUMMER>>.
Könnten Sie mir bitte erklären, wie sich der monatliche Abschlag zusammensetzt? Ich bin mir nicht sicher, was die einzelnen Posten auf meiner Rechnung bedeuten.
Haben Sie außerdem eine Empfehlung, wie ich meinen Verbrauch besser kontrollieren und möglicherweise verringern kann?
Ich freue mich auf Ihre Antwort.
Vielen Dank und beste Grüße,
<<Michael Schäfer&&PERSON>>





Input:
Sehr geehrte Damen und Herren,

ich bin Kunde bei E.ON mit der Vertragsnummer 232456789123 und habe eine Frage bezüglich meines Kundenkontos. Wie kann ich einen neuen Kundenberater beantragen? Ich wäre Ihnen sehr dankbar, wenn Sie mir diese Information zukommen lassen könnten.

Vielen Dank im Voraus für Ihre Hilfe.

Mit freundlichen Grüßen,

Alexander Schmidt
Am Musterweg 4
12345 Musterstadt
alex.schmidt@mustermail.de
Tel: 0176 12345678

Output:

Sehr geehrte Damen und Herren,
ich bin Kunde bei E.ON mit der Vertragsnummer <<232456789123&&VERTRAGSNUMMER>> und habe eine Frage bezüglich meines Kundenkontos. Wie kann ich einen neuen Kundenberater beantragen? Ich wäre Ihnen sehr dankbar, wenn Sie mir diese Information zukommen lassen könnten.
Vielen Dank im Voraus für Ihre Hilfe.
Mit freundlichen Grüßen,
<<Alexander Schmidt&&PERSON>>
<<Am Musterweg 4&&STRASSE>>
<<12345&&PLZ>><<Musterstadt&&ORT>>
<<alex.schmidt@mustermail.de&&EMAIL>>
Tel: <<0176 12345678&&TELEFONNUMMER>>



Input
Guten Tag,


mein Name ist Sabine Müller und ich bin Kunde bei E.ON mit der Vertragsnummer 401234567. Ich habe eine Frage bezüglich des Kundenservice. Vor einigen Tagen hatte ich ein Problem mit meiner Stromversorgung und wollte Kontakt mit einem Kundenberater aufnehmen. Leider konnte ich niemanden erreichen.


Könnten Sie mir bitte mitteilen, wie ich am besten einen Kundenbetreuer erreichen kann, falls ich zukünftig wieder auf Probleme stoße? Gibt es eine spezielle Telefonnummer oder E-Mail-Adresse, die ich verwenden kann? Oder kann ich auch über das Kundenportal auf Ihrer Webseite einen Kundenberater kontaktieren?


Ich würde mich über eine schnelle Rückmeldung freuen.


Vielen Dank im Voraus.


Mit freundlichen Grüßen,
Sabine Müller
Sabine.Mueller@gmail.com
Am Sonnenhang 34, 12345 Musterstadt
Tel: 0176 12345678

Output
Guten Tag,
mein Name ist <<Sabine Müller&&PERSON>> und ich bin Kunde bei E.ON mit der Vertragsnummer <<401234567&&VERTRAGSNUMMER>>. Ich habe eine Frage bezüglich des Kundenservice. Vor einigen Tagen hatte ich ein Problem mit meiner Stromversorgung und wollte Kontakt mit einem Kundenberater aufnehmen. Leider konnte ich niemanden erreichen.
Könnten Sie mir bitte mitteilen, wie ich am besten einen Kundenbetreuer erreichen kann, falls ich zukünftig wieder auf Probleme stoße? Gibt es eine spezielle Telefonnummer oder E-Mail-Adresse, die ich verwenden kann? Oder kann ich auch über das Kundenportal auf Ihrer Webseite einen Kundenberater kontaktieren?
Ich würde mich über eine schnelle Rückmeldung freuen.
Vielen Dank im Voraus.
Mit freundlichen Grüßen,
<<Sabine Müller&&PERSON>>
<<Sabine.Mueller@gmail.com&&EMAIL>>
<<Am Sonnenhang 34&&STRASSE>>, <<12345&&PLZ>>,
 <<Musterstadt&&ORT>>
Tel: <<0176 12345678&&TELEFONNUMMER>>

"""



#3. Run all Scripts from here: Start Anonymization

##load all texts

In [13]:
#load data from google drive and save to numpy array(filename,string)
import os
import numpy as np
def load_text_files_to_numpy_with_filenames(folder_path):
    # Initialize an empty list to store text contents along with file names
    texts_with_filenames = []

    # Walk through all files in the specified folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a text file
        if file_name.endswith('.txt'):
            # Construct full file path
            file_path = os.path.join(folder_path, file_name)
            # Open and read the content of the text file
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Append tuple of file name and content
                texts_with_filenames.append((file_name, content))

    # Convert the list to a numpy array of tuples
    texts_with_filenames_np = np.array(texts_with_filenames, dtype=object)
    return texts_with_filenames_np


folder_path = folder_path #defined at the setup section
loaded_texts = load_text_files_to_numpy_with_filenames(folder_path)

# Debugging:
#loaded_texts[0][0]
#type(loaded_texts)



In [14]:
#debugging
#len(loaded_texts)

###Prepare Data

In [15]:
#extract list of strings from loaded_texts array (filename, inputstring)
def extract_input_strings(loaded_texts):
    # Initialize the resulting list of strings
    resulting_list = []

    # Loop starting from the second element (index 1)
    for i in range(0, len(loaded_texts)):
        # Append the items from the sublist at index 1
        # Assuming each element at index 1 is an array-like structure containing strings
        resulting_list.append(loaded_texts[i][1])

    return resulting_list

# The call to the function is commented out as per instructions
list_of_input_strings= extract_input_strings(loaded_texts)
#len(list_of_input_strings)

In [16]:
#extract list of filenames from tuple (filename, inputstring)
def extract_input_filename(loaded_texts):
    # Initialize the resulting list of strings
    resulting_list = []

    # Loop starting from the second element (index 1)
    for i in range(0, len(loaded_texts)):
        # Append the items from the sublist at index 1
        # Assuming each element at index 1 is an array-like structure containing strings
        resulting_list.append(loaded_texts[i][0])

    return resulting_list

# The call to the function is commented out as per instructions
list_of_input_filenames= extract_input_filename(loaded_texts)
#list_of_input_filenames

## LM Studio API Call

In [19]:
from openai import OpenAI
model1="TheBloke/SauerkrautLM-Mixtral-8x7B-Instruct-GGUF"

In [ ]:
# dont change here, if you want all text to be scanned
num_of_texts_to_scan = 3 #len(loaded_texts) # num of chatgpt request iterations musst be same lenght as truth json

# Configuration for OpenAI client
# Point to the local server
# use ip route to get the right ip address
client = OpenAI(base_url="http://localhost:4502/v1", api_key="lm-studio")


def gpt_request(text_input):
    # Assuming system_prompt is predefined somewhere in your code
    response = client.chat.completions.create(
        model=model1,
        temperature = 0,
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text_input}],
        seed=42)
    return response.choices[0].message.content


def process_texts(texts):
    responses = []
    # Wrap the loop with tqdm for a progress bar
    for text in tqdm.tqdm(texts[:num_of_texts_to_scan], desc="Processing texts"):
        response = gpt_request(text)
        responses.append(response)
    # Convert the list of responses into a NumPy array
    response_array = np.array(responses)
    return response_array


response_array = process_texts(list_of_input_strings)


Processing texts:   0%|                                                                          | 0/3 [00:00<?, ?it/s]

In [ ]:
#for debugging
#response_array[1]



##Prepare output data


In [ ]:
import pandas as pd

def create_dataframe_from_lists(response_array, list_of_input_strings, list_of_input_filenames):
    # Find the minimum length to slice all lists equally without index errors
    min_length = min(len(response_array), len(list_of_input_strings), len(list_of_input_filenames), 1010)

    # Creating a dictionary where each sliced list becomes a column in the DataFrame
    data = {
        'output': response_array[:min_length],
        'input': list_of_input_strings[:min_length],
        'pred_filename': list_of_input_filenames[:min_length],
        'system_prompt': system_prompt
    }

    # Creating the DataFrame using the dictionary
    df = pd.DataFrame(data)
    return df


df = create_dataframe_from_lists(response_array, list_of_input_strings, list_of_input_filenames)
df.head()


## create prediction_dict

In [ ]:
#create a new column and create a dict with entity_type, and entitiy_value
#find values based on pattes


# Function to extract labeled values
def extract_labeled_values(text):
    # Regex to match content between << and >>
    pattern = r"<<(.*?)&&(.*?)>>"

    # Find all matches in the text
    matches = re.findall(pattern, text)

    # Create a list of dictionaries from the matches
    label_value_list = [{'entity_type': match[1], 'entity_value': match[0]} for match in matches]

    return label_value_list

# Create a new column with the extracted label-value list
df['pred_dict'] = df['output'].apply(extract_labeled_values)

#df.head()

### calculate START index of predicted entity

In [ ]:
#find start and stop index of each entity

#find each pred_value starting index
import pandas as pd
import re

def update_unique_pred_start_indices(data, text):
    # Check if data is empty, return an empty dict immediately
    if not data:
        return data

    # Dictionary to hold all matches for each value
    all_matches = {}
    # Dictionary to track the next available index for each value
    next_available_index = {}

    # First, find all matches for each unique value
    for item in data:
        value = item['entity_value']
        if value not in all_matches:
            regex = r'\b' + re.escape(value) + r'\b'
            all_matches[value] = [match.start() for match in re.finditer(regex, text)]
            next_available_index[value] = 0  # Initialize the index tracker for each unique value

    # Next, assign the first available match index to each item in data
    for item in data:
        value = item['entity_value']
        matches = all_matches[value]
        index = next_available_index[value]

        if index < len(matches):
            item['start'] = matches[index]
            next_available_index[value] += 1  # Move to the next index for the next occurrence
        else:
            # Optionally handle the case where there are more items than matches (not expected)
            item['start'] = None

    return data

# Example DataFrame and applying the function
# df = pd.DataFrame({'dict_start': [[{'pred_value': 'example'}]], 'text': ['This is an example text for example usage.']})
df['dict_start_index'] = df.apply(lambda row: update_unique_pred_start_indices(row['pred_dict'], row['input']), axis=1)
#df['dict_start_index'].iloc[0]

### calc END index of predicted entity

In [ ]:
#calculate end index of predicted value

def add_pred_end_to_dicts(pred_dicts):
    """ Function to add 'pred_end' key to each dictionary in a list, calculated from 'pred_start' and the length of 'pred_value'. """
    updated_dicts = []
    for entry in pred_dicts:
        if 'start' in entry and entry['start'] is not None:  # Ensure 'pred_start' exists and is not None
            entry['end'] = entry['start'] + len(entry['entity_value'])
        updated_dicts.append(entry)
    return updated_dicts

# Assuming df is your DataFrame and 'dict_start' is the column with lists of dictionaries
df['pred_dict_start_end'] = df['dict_start_index'].apply(lambda x: add_pred_end_to_dicts(x) if x is not None else None)

# Now, let's see the updated DataFrame
#df['pred_dict_start_end'].iloc[0]


### extract a dictionary from the df with a list of dicts



In [ ]:

# Create the new list of dictionaries with filename and list of entities
output_dict= []
for _, row in df.iterrows():
    new_dict = {
        row['pred_filename'] : row['pred_dict_start_end'],
    }
    output_dict.append(new_dict)

#output_dict[1:2]  # New list of dictionaries with filename and entity data

In [ ]:
resulting_dict = {}

# Iterate over each dictionary in the list
for data in output_dict:
    for key, value in data.items():
        # Add the key and its corresponding list of dictionaries to the output dictionary
        resulting_dict[key] = value

#resulting_dict

##save output to folder .txt and .json

In [ ]:
import os
import json
from datetime import datetime
import pandas as pd

def dataframe_to_text_files_in_drive(df, output_folder):
    txt_folder_path = os.path.join(output_folder, 'txt')

    # Ensure the necessary columns exist
    if 'pred_filename' not in df.columns or 'output' not in df.columns:
        raise ValueError("DataFrame must contain 'pred_filename' and 'output' columns.")

    # Create the txt subdirectory if it does not exist
    os.makedirs(txt_folder_path, exist_ok=True)

    for index, row in df.iterrows():
        # Construct the full path for the file within the txt subdirectory
        full_path = os.path.join(txt_folder_path, f"{row['pred_filename']}")

        # Write the output to the file
        with open(full_path, 'w', encoding='utf-8') as file:
            file.write(str(row['output']))

def save_predictions_as_json(predictions_dict, output_folder):
    json_folder_path = os.path.join(output_folder, 'json')

    # Create the json subdirectory if it does not exist
    os.makedirs(json_folder_path, exist_ok=True)

    # Get the current datetime and format it for file naming
    current_dt = datetime.now().strftime("%Y-%m-%d_%H-%M")
    filename = f"predictions_dict_{current_dt}.json"
    full_path = os.path.join(json_folder_path, filename)

    # Save predictions dictionary as a JSON file
    with open(full_path, 'w') as file:
        json.dump(predictions_dict, file)

# Get the current datetime and create a timestamped subdirectory
current_dt = datetime.now().strftime("%Y-%m-%d_%H-%M")
output_folder = os.path.join(main_output_folder, current_dt)
os.makedirs(output_folder, exist_ok=True)

# Call functions with the correct folder
dataframe_to_text_files_in_drive(df, output_folder)
save_predictions_as_json(resulting_dict, output_folder)
